In [1]:
#####マーケットデータのスクレイピング#####
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from sqlalchemy import create_engine
from urllib.request import urlopen
import pandas as pd
import numpy as np
import os
import time
import itertools
import MySQLdb as sql
import mysql.connector
import requests

In [25]:
####為替レートの取得とDBへの保存####
##為替レートのクレンジング
#為替レートを取得
exchange_df = pd.read_csv("https://www.mizuhobank.co.jp/market/csv/ref.csv", encoding="Shift-Jis", header=2)
exchange_df.columns = ["date", "USD1", "GBP1", "EUR1", "CAD1", "AUD1", "NZD1", "CNH1", "HKD1", "SGD1", "CHF1", "NOK1", "SEK1", 
                       "USD2", "GBP2", "EUR2", "CAD2", "AUD2", "NZD2", "CNH2"]

#文字列型を数値型に変更
columns = ["USD1", "GBP1", "EUR1", "CAD1", "AUD1", "NZD1", "CNH1", "HKD1", "SGD1", "CHF1", "NOK1", "SEK1", 
           "USD2", "GBP2", "EUR2", "CAD2", "AUD2", "NZD2", "CNH2"]
for j in range(len(columns)):
    if exchange_df.iloc[:, j+1].dtype=="float64":
        exchange_df[columns[j]] = exchange_df[columns[j]].astype("float16")
    else:
        exchange_df[columns[j]] = exchange_df[columns[j]].astype("str").replace("*****", "NaN").astype("float16")
        
#日付の形式を変更
target_date = exchange_df["date"]
date = np.array(np.repeat("", target_date.shape[0]), dtype="U10")
temp_date = target_date.str.split("/")
for i in range(temp_date.shape[0]):
    for j in range(1, 3):
        if len(temp_date[i][j])==1:
            temp_date[i][j] = "0" + temp_date[i][j]
    date[i] = temp_date[i][0] + "-" + temp_date[i][1] + "-" + temp_date[i][2]
exchange_df["date"] = date


##DBの作成とデータの格納
#DBを作成して、適当なデータをMySQLに格納
password = open("password.txt").read()
connect = sql.connect(db="economic_indicators", user="root", passwd=password, host="localhost", charset="utf8mb4")
c = connect.cursor()   #カーソルを取得

#新しいテーブルを作成
c.execute("DROP TABLE IF EXISTS exchange_rate_log")   #テーブルがある場合、テーブルを削除
c.execute("CREATE TABLE exchange_rate_log\
          (\
            date varchar(10),\
            USD1 float(5, 5),\
            GBP1 float(5, 5),\
            EUR1 float(5, 5),\
            CAD1 float(5, 5),\
            AUD1 float(5, 5),\
            NZD1 float(5, 5),\
            CNH1 float(5, 5),\
            HKD1 float(5, 5),\
            SGD1 float(5, 5),\
            CHF1 float(5, 5),\
            NOK1 float(5, 5),\
            SEK1 float(5, 5),\
            USD2 float(5, 5),\
            GBP2 float(5, 5),\
            EUR2 float(5, 5),\
            CAD2 float(5, 5),\
            AUD2 float(5, 5),\
            NZD2 float(5, 5),\
            CNH2 float(5, 5))")

#DBに接続
db_settings = {
    "host": "localhost",
    "database": "economic_indicators",
    "user": "root",
    "password": password,
    "port": 3306
}
con = mysql.connector.connect(**db_settings)
engine = create_engine('mysql://{user}:{password}@{host}:{port}/{database}'.format(**db_settings))

#DBに格納
exchange_df.to_sql("exchange_rate_log", engine, index=False, if_exists="replace")

In [20]:
####国内の株価指標の取得とDBへの格納####
